# **03_data_api_processing_merge.ipynb**

В этом ноутбуке данные были обработаны и объединены с основной таблицей объектов, собранной при парсинге Move.ru.

In [ ]:
import requests
import json
import time
from typing import List, Dict, Any
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import numpy as np

------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
dff = pd.read_csv('итогAPI.csv')

РАБОТА С ПРОПУСКАМИ

In [ ]:
dff

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
0,Академический,бизнес-центр,"Ферро-Плаза, бизнес-центр","улица Кржижановского, 14 к3",55.679541,37.566590,3.7,8,30
1,Академический,бизнес-центр,"Микроэкономика, бизнес-центр","Большая Черёмушкинская улица, 34",55.679174,37.583232,3.7,5,36
2,Академический,бизнес-центр,"iCube, бизнес-центр","Нахимовский проспект, 58",55.683738,37.554129,4.7,11,36
3,Академический,бизнес-центр,"Академический, бизнес-центр","60-летия Октября проспект, 9 ст2",55.698817,37.580293,нет рейтинга,0,27
4,Академический,бизнес-центр,Бизнес-Недвижимость,"Новочерёмушкинская улица, 25",55.680746,37.578958,4.9,2,13
...,...,...,...,...,...,...,...,...,...
21213,Ясенево,школа,"Школа №1103, школа им. Героя РФ А.В. Соломатина","Литовский бульвар, 42 к2",55.608020,37.541734,4.9,1,4
21214,Ясенево,школа,Школа №2103,"Голубинская улица, 13 к2",55.603158,37.517547,нет рейтинга,0,2
21215,Ясенево,школа,"Школа им. Н.М. Карамзина, Остафьевский корпус №9","проезд Карамзина, 9 к2",55.591577,37.540696,нет рейтинга,0,0
21216,Ясенево,школа,"London Express, школа","Соловьиный проезд, 1",55.619451,37.543880,5,3,5


In [ ]:
dff.isna().sum()

,0
район,0
тип,0
название,0
адрес,190
широта,0
долгота,0
рейтинг,0
количество_отзывов,0
количество_оценок,0


Видим, что пропуски есть только в признаке адрес. Это связано с тем, что на сайте 2гис для некоторых объектов не указан адрес. У таких объектов указан только район и город.

In [ ]:
dff[dff['адрес'].isna()]

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
239,Алексеевский,кафе,"Церковный дворик, кафе фермерской продукции",NaN,55.808282,37.654088,нет рейтинга,0,1
562,Арбат,музей,"Музей счастливого детства, книжная лавка",NaN,55.752312,37.584955,нет рейтинга,0,4
748,Аэропорт,парк,Петровский парк,NaN,55.793866,37.557141,4.7,99,669
1151,Басманный,парк,"Горка, парк",NaN,55.756114,37.636096,4.2,1,3
1623,Бибирево,кафе,"Piazza del cafе, кофейный автомат",NaN,55.883405,37.604799,нет рейтинга,0,0
...,...,...,...,...,...,...,...,...,...
20865,Якиманка,парк,Центральный парк культуры и отдыха им. М. Горь...,NaN,55.728201,37.600249,4.9,1891,7808
20866,Якиманка,парк,"Нескучный сад, парк",NaN,55.717043,37.586490,4.8,327,1377
20867,Якиманка,парк,"Музеон, парк искусств",NaN,55.736473,37.608145,4.9,477,2132
21159,Ясенево,парк,"Принц парк отель, Парковка ГК Принц парк отель",NaN,55.616813,37.511657,нет рейтинга,0,0


In [ ]:
dff['рейтинг'].value_counts()

,count
рейтинг,
нет рейтинга,5256
5,3655
4.7,1475
4.8,1444
4.6,1132
4.9,1103
4.5,984
4.4,654
4.3,564


Однако еще мы можем понять, что пропуски есть и в признаке 'рейтинг', когда рейтинг на сайте 2гис отсутствует. У нас в признаке это помечается как "нет рейтинга", поэтому в пропуски это не суммируется, так как значение в признаке присутствует. Тогда заменим все значения "нет рейтинга" на NaN.

In [ ]:
import re

In [ ]:
dff.loc[dff['рейтинг'].str.match(r'^нет рейтинга$', na=False), 'рейтинг'] = np.nan

In [ ]:
dff

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
0,Академический,бизнес-центр,"Ферро-Плаза, бизнес-центр","улица Кржижановского, 14 к3",55.679541,37.566590,3.7,8,30
1,Академический,бизнес-центр,"Микроэкономика, бизнес-центр","Большая Черёмушкинская улица, 34",55.679174,37.583232,3.7,5,36
2,Академический,бизнес-центр,"iCube, бизнес-центр","Нахимовский проспект, 58",55.683738,37.554129,4.7,11,36
3,Академический,бизнес-центр,"Академический, бизнес-центр","60-летия Октября проспект, 9 ст2",55.698817,37.580293,NaN,0,27
4,Академический,бизнес-центр,Бизнес-Недвижимость,"Новочерёмушкинская улица, 25",55.680746,37.578958,4.9,2,13
...,...,...,...,...,...,...,...,...,...
21213,Ясенево,школа,"Школа №1103, школа им. Героя РФ А.В. Соломатина","Литовский бульвар, 42 к2",55.608020,37.541734,4.9,1,4
21214,Ясенево,школа,Школа №2103,"Голубинская улица, 13 к2",55.603158,37.517547,NaN,0,2
21215,Ясенево,школа,"Школа им. Н.М. Карамзина, Остафьевский корпус №9","проезд Карамзина, 9 к2",55.591577,37.540696,NaN,0,0
21216,Ясенево,школа,"London Express, школа","Соловьиный проезд, 1",55.619451,37.543880,5,3,5


In [ ]:
dff.isna().sum()

,0
район,0
тип,0
название,0
адрес,190
широта,0
долгота,0
рейтинг,5256
количество_отзывов,0
количество_оценок,0


Таким образом, в датасете после апи-парсинга в признаке 'рейтинг' 190 пропусков из 21218, а в признаке 'рейтинг' - 5256 из 2128.

In [ ]:
dff[(dff['рейтинг'].isna()) & (dff['количество_оценок']!=0)]

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
3,Академический,бизнес-центр,"Академический, бизнес-центр","60-летия Октября проспект, 9 ст2",55.698817,37.580293,NaN,0,27
5,Академический,бизнес-центр,"Принципал Плаза, бизнес-центр","60-летия Октября проспект, 10а",55.692721,37.574555,NaN,0,38
9,Академический,бизнес-центр,Бизнес-центр,"улица Ивана Бабушкина, 16а",55.684838,37.560352,NaN,0,1
10,Академический,бизнес-центр,"Карьер, бизнес-центр","улица Карьер, 2а ст2",55.694974,37.591161,NaN,0,1
11,Академический,бизнес-центр,"Plaza Kedrova, офисный центр","улица Кедрова, 13 к2",55.684730,37.572909,NaN,0,1
...,...,...,...,...,...,...,...,...,...
21204,Ясенево,школа,"Школа №1561, корпус №4","Новоясеневский проспект, 38 к2",55.602029,37.543707,NaN,0,2
21206,Ясенево,школа,Школа №2103,"Голубинская улица, 5 к4",55.604795,37.517975,NaN,0,4
21208,Ясенево,школа,"Ясенево, школа №1694 им. П.М. Фитина","Соловьиный проезд, 4 к2",55.614998,37.547954,NaN,0,2
21209,Ясенево,школа,Школа №1212,"Новоясеневский проспект, 24 к3",55.603855,37.528777,NaN,0,1


In [ ]:
dff[dff['рейтинг'].isna() & dff['количество_отзывов']!=0]

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок


In [ ]:
dff[dff['рейтинг'].notna()]['количество_отзывов'].value_counts()

,count
количество_отзывов,
1,2409
2,1499
3,997
4,723
5,516
...,...
1199,1
671,1
509,1


Видно, что у всех объектов, у которых есть рейтинг, число отзывов больше или равно 1.

In [ ]:
dff[dff['рейтинг'].isna()]['количество_отзывов'].value_counts()

,count
количество_отзывов,
0,5256


А у всех объетков, у которых отсутствует рейтинг, число отзывов равно 0.

Таким образом, пропуски в признаке 'рейтинг' появились из-за того, что 2ГИС не показывает рейтинг, пока не наберется минимальное количество отзывов - хотя бы 1 отзыв.

+ Новые организации, которые недавно добавились в каталог и не получили отзывов
+ Компании, которые запретили отзывы
+ Непопулярные и малоизвестные организации без отзывов

________________________________________________________________________________________________________________

Обработка датасета для склеивания с веб-скрапингом.

In [ ]:
res = pd.read_csv('количество.csv')

In [ ]:
res

,район,бизнес-центр,детский сад,кафе,магазин,музей,парк,поликлиника,торговый центр,школа
0,Академический,13,30,38,50,8,1,9,2,19
1,Алексеевский,17,21,44,50,2,0,9,3,16
2,Алтуфьевский,7,10,5,32,1,1,4,2,6
3,Арбат,27,11,50,50,27,1,5,8,9
4,Аэропорт,21,32,50,50,7,12,9,7,15
...,...,...,...,...,...,...,...,...,...,...
127,Южное Тушино,11,19,24,50,0,2,3,4,22
128,Южнопортовый,18,19,50,50,1,8,10,9,15
129,Якиманка,26,9,50,47,19,3,6,4,13
130,Ярославский,4,23,22,50,0,1,4,8,14


In [ ]:
res.isna().sum()

,0
район,0
бизнес-центр,0
детский сад,0
кафе,0
магазин,0
музей,0
парк,0
поликлиника,0
торговый центр,0
школа,0


В данном датасете нет пропусков, что логично, так как при группировке и сборе этого датасета (по таблице итогAPI) для каждого района считалось число имеющихся в нем объектов разного типа, и поэтому для всех типов в каждом регионе есть значение количества этого объекта в регионе.

________________________________________________________________________________________________________________

Таблица с информацией по районам Москвы из викепедии. Информация в таблице из Росстата, а у 8 из 132 районов нет информации, так как они были созданы только в 2024 году и на Росстате нет информации о них.

In [ ]:
regions = pd.read_excel('wikip.xlsx')

In [ ]:
regions

,Название района,Административный округ,"Площадь, км²","Население на 2024, чел.","Плотность населения на 2024, чел./км²","Площадь жилого фонда (01.01.2010), тыс. м²","Жилплощадь на человека (01.01.2010), м²/чел."
0,Академический,ЮЗАО,5.83,114 228,19593.14,2467,22.7
1,Алексеевский,СВАО,5.29,77 681,14684.5,1607.9,20.5
2,Алтуфьевский,СВАО,3.25,55 741,17151.08,839.3,15.5
3,Арбат,ЦАО,2.11,36 246,17178.2,731,26
4,Аэропорт,САО,4.58,81 833,17867.47,1939.7,25.9
...,...,...,...,...,...,...,...
127,Южное Тушино,СЗАО,7.94,112 628,14184.89,1714.9,16.9
128,Южнопортовый,ЮВАО,4.53,73 112,16139.51,1449,20.6
129,Якиманка,ЦАО,4.8,27 022,5629.58,788.7,29.8
130,Ярославский,СВАО,7.99,98 861,12373.09,1404.5,15.3


In [ ]:
regions[regions['Название района']=='Бекасово']

,Название района,Административный округ,"Площадь, км²","Население на 2024, чел.","Плотность населения на 2024, чел./км²","Площадь жилого фонда (01.01.2010), тыс. м²","Жилплощадь на человека (01.01.2010), м²/чел."
8,Бекасово,ТАО,—,Неизвестно,Неизвестно,Неизвестно,Неизвестно


In [ ]:
import re

In [ ]:
regions['Площадь, км²'] = regions['Площадь, км²'].replace(r'^—$', np.nan, regex=True)

In [ ]:
regions = regions.replace(r'^Неизвестно$', np.nan, regex=True)

/tmp/ipython-input-4117153067.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  regions = regions.replace('Неизвестно', np.nan)


In [ ]:
regions.isna().sum()

,0
Название района,0
Административный округ,0
"Площадь, км²",8
"Население на 2024, чел.",8
"Плотность населения на 2024, чел./км²",8
"Площадь жилого фонда (01.01.2010), тыс. м²",7
"Жилплощадь на человека (01.01.2010), м²/чел.",7


Заполним информацией с другого сайта (энциклопедия).

Таблица с заполненными пропусками будет в файле wikipWITHOUT.xlsx

________________________________________________________________________________________________________________

Соединим таблицы, полученные в ходе веб-скрапинга и апи-парсинга.

In [ ]:
pars = pd.read_csv('parsingMOVEsplit.csv')

In [ ]:
pars['район'] = pars['district'].str.replace('район', '', regex=False)
# Убираем лишние пробелы, которые могли образоваться
pars['район'] = pars['район'].str.strip()
del pars['district']

In [ ]:
pars

,price_millions,complex_name,developer,housing_class,floor,total_floors,rooms,total_area,living_area,kitchen_area,...,metro_data,metro_names,nearest_metro,nearest_metro_distance,views,url,city,okrug,other,район
0,629.32,Клубный дом «Хамовники 12»,COLDY,Элит класс,2.0,14.0,4.0,202.70,140.00,20.0,...,"[{'name': 'Киевская', 'distance_m': 1000}, {'n...","['Киевская', 'Фрунзенская', 'Парк культуры', '...",Киевская,1000.0,320.0,https://move.ru/objects/moskva_pereulok_trujen...,Москва,ЦАО,"пер. 1-й Тружеников, 12",Хамовники
1,25.18,Квартал «Серебряный бор»,StMichael,Бизнес класс,7.0,25.0,NaN,24.07,14.56,NaN,...,"[{'name': 'Щукинская', 'distance_m': 3000}, {'...","['Щукинская', 'Октябрьское поле', 'Народное Оп...",Щукинская,3000.0,61.0,https://move.ru/objects/prodaetsya_studiya_plo...,Москва,СЗАО,"ул. Берзарина, 3",Щукино
2,20.27,Апарт-комплекс «Движение.Тушино»,ГК «ФСК»,Комфорт класс,19.0,22.0,2.0,52.80,26.40,11.5,...,"[{'name': 'Спартак', 'distance_m': 682}, {'nam...","['Спартак', 'Тушинская', 'Щукинская', 'Строгин...",Спартак,682.0,64.0,https://move.ru/objects/prodaetsya_2-komnatnay...,Москва,СЗАО,"Волоколамское шоссе, 4",Покровское-Стрешнево
3,12.61,ЖК «BeLuck Коломенское»,ГК «Гранель»,Комфорт класс,16.0,32.0,NaN,32.36,23.56,NaN,...,"[{'name': 'Нагатинская', 'distance_m': 1000}, ...","['Нагатинская', 'Нагорная', 'Коломенская', 'На...",Нагатинская,1000.0,64.0,https://move.ru/objects/prodaetsya_studiya_plo...,Москва,ЮАО,"Каширское шоссе, 1.1",Нагатино-Садовники
4,14.11,ЖК VEER,MR Group,Бизнес класс,6.0,22.0,NaN,22.20,14.35,NaN,...,"[{'name': 'Давыдково', 'distance_m': 2000}, {'...","['Давыдково', 'Аминьевская', 'Кунцевская', 'Сл...",Давыдково,2000.0,56.0,https://move.ru/objects/prodaetsya_studiya_plo...,Москва,ЗАО,"ул. Багрицкого, Indigo-4",Можайский
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10524,21.76,ЖК Moments,FORMA,Бизнес класс,5.0,30.0,1.0,44.50,14.40,21.4,...,"[{'name': 'Войковская', 'distance_m': 2000}, {...","['Войковская', 'Щукинская', 'Октябрьское поле'...",Войковская,2000.0,19.0,https://move.ru/objects/prodaetsya_1-komnatnay...,Москва,СЗАО,"1-й Красногорский проезд, Корпус 2.1",Щукино
10525,17.55,ЖК «Руставели 14»,ПИК,Комфорт класс,3.0,21.0,1.0,31.80,9.40,15.0,...,"[{'name': 'Бутырская', 'distance_m': 524}, {'n...","['Бутырская', 'Фонвизинская', 'Дмитровская', '...",Бутырская,524.0,5.0,https://move.ru/objects/prodaetsya_1-komnatnay...,Москва,СВАО,Корпус 3.7,Бутырский
10526,20.08,NaN,NaN,NaN,11.0,43.0,NaN,30.10,13.10,NaN,...,"[{'name': 'Крылатское', 'distance_m': 3000}, {...","['Крылатское', 'Молодежная', 'Народное Ополчен...",Крылатское,3000.0,28.0,https://move.ru/objects/prodaetsya_studiya_plo...,Москва,ЗАО,"ул. Крылатская, 33",Крылатское
10527,22.05,ЖК Moments,FORMA,Бизнес класс,24.0,34.0,1.0,43.40,14.80,20.8,...,"[{'name': 'Войковская', 'distance_m': 2000}, {...","['Войковская', 'Щукинская', 'Октябрьское поле'...",Войковская,2000.0,6.0,https://move.ru/objects/prodaetsya_1-komnatnay...,Москва,СЗАО,"1-й Красногорский проезд, Корпус 2.2",Щукино


In [ ]:
merged = pars.merge(res, how='left', on = 'район')

In [ ]:
merged.to_csv('merged.csv')

In [ ]:
merged

,price_millions,complex_name,developer,housing_class,floor,total_floors,rooms,total_area,living_area,kitchen_area,...,район,бизнес-центр,детский сад,кафе,магазин,музей,парк,поликлиника,торговый центр,школа
0,629.32,Клубный дом «Хамовники 12»,COLDY,Элит класс,2.0,14.0,4.0,202.70,140.00,20.0,...,Хамовники,45,43,50,50,39,1,20,1,36
1,25.18,Квартал «Серебряный бор»,StMichael,Бизнес класс,7.0,25.0,NaN,24.07,14.56,NaN,...,Щукино,7,35,43,50,3,0,15,8,24
2,20.27,Апарт-комплекс «Движение.Тушино»,ГК «ФСК»,Комфорт класс,19.0,22.0,2.0,52.80,26.40,11.5,...,Покровское-Стрешнево,12,18,42,50,3,4,9,7,14
3,12.61,ЖК «BeLuck Коломенское»,ГК «Гранель»,Комфорт класс,16.0,32.0,NaN,32.36,23.56,NaN,...,Нагатино-Садовники,16,23,50,50,1,2,10,6,27
4,14.11,ЖК VEER,MR Group,Бизнес класс,6.0,22.0,NaN,22.20,14.35,NaN,...,Можайский,27,42,44,50,0,23,6,8,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10524,21.76,ЖК Moments,FORMA,Бизнес класс,5.0,30.0,1.0,44.50,14.40,21.4,...,Щукино,7,35,43,50,3,0,15,8,24
10525,17.55,ЖК «Руставели 14»,ПИК,Комфорт класс,3.0,21.0,1.0,31.80,9.40,15.0,...,Бутырский,21,17,50,50,2,7,4,10,11
10526,20.08,NaN,NaN,NaN,11.0,43.0,NaN,30.10,13.10,NaN,...,Крылатское,6,18,24,50,2,3,13,11,14
10527,22.05,ЖК Moments,FORMA,Бизнес класс,24.0,34.0,1.0,43.40,14.80,20.8,...,Щукино,7,35,43,50,3,0,15,8,24


Теперь присоединим данные про районы.

In [ ]:
wik = pd.read_excel('wikipWITHOUT.xlsx')

In [ ]:
merged = merged.merge(wik, how = 'left', on = 'район')

Целевой переменной нашего проекта будет являтся цена квадратного метра. Добавим этот признак.

In [ ]:
merged['цена_за_квМ'] = merged['price_millions'] / merged['total_area']

In [ ]:
merged.to_csv('merge.csv')

Теперь добавим признаки, которые будут показывать средний рейтинг по определенному объекту в определенном районе.

In [ ]:
dff = dff.dropna(subset=['рейтинг']) #удаляем строки с пропусками в столбце рейтинг, чтобыдальше они не мешали при расчетах

In [ ]:
dff['рейтинг'] = dff['рейтинг'].astype(float)

In [ ]:
gr = dff.groupby(['район', 'тип'])['рейтинг'].mean().unstack()

In [ ]:
gr.columns = [f'средний_рейтинг_{str(col).lower()}_в_районе' for col in gr.columns]

In [ ]:
gr

,средний_рейтинг_бизнес-центр_в_районе,средний_рейтинг_детский сад_в_районе,средний_рейтинг_кафе_в_районе,средний_рейтинг_магазин_в_районе,средний_рейтинг_музей_в_районе,средний_рейтинг_парк_в_районе,средний_рейтинг_поликлиника_в_районе,средний_рейтинг_торговый центр_в_районе,средний_рейтинг_школа_в_районе
район,,,,,,,,,
Академический,4.542857,4.314286,4.244444,4.177083,4.442857,4.000000,3.475000,4.350000,4.175000
Алексеевский,4.200000,4.500000,4.282051,4.522000,4.950000,NaN,3.166667,4.800000,4.400000
Алтуфьевский,4.880000,4.500000,3.900000,4.348000,NaN,3.800000,4.466667,4.800000,4.380000
Арбат,4.716667,5.000000,4.151111,4.314286,4.760000,4.900000,4.733333,4.562500,4.737500
Аэропорт,4.668750,4.871429,4.280952,4.329545,4.883333,3.960000,3.642857,4.483333,4.577778
...,...,...,...,...,...,...,...,...,...
Южное Тушино,4.550000,4.320000,3.955000,4.368889,NaN,NaN,3.850000,4.650000,4.526667
Южнопортовый,4.250000,4.583333,3.922500,4.457447,5.000000,4.600000,4.220000,4.500000,4.361538
Якиманка,4.678571,5.000000,4.197959,4.408333,4.582353,4.866667,4.400000,4.650000,4.518182


In [ ]:
gr = gr.reset_index()
gr

,район,средний_рейтинг_бизнес-центр_в_районе,средний_рейтинг_детский сад_в_районе,средний_рейтинг_кафе_в_районе,средний_рейтинг_магазин_в_районе,средний_рейтинг_музей_в_районе,средний_рейтинг_парк_в_районе,средний_рейтинг_поликлиника_в_районе,средний_рейтинг_торговый центр_в_районе,средний_рейтинг_школа_в_районе
0,Академический,4.542857,4.314286,4.244444,4.177083,4.442857,4.000000,3.475000,4.350000,4.175000
1,Алексеевский,4.200000,4.500000,4.282051,4.522000,4.950000,NaN,3.166667,4.800000,4.400000
2,Алтуфьевский,4.880000,4.500000,3.900000,4.348000,NaN,3.800000,4.466667,4.800000,4.380000
3,Арбат,4.716667,5.000000,4.151111,4.314286,4.760000,4.900000,4.733333,4.562500,4.737500
4,Аэропорт,4.668750,4.871429,4.280952,4.329545,4.883333,3.960000,3.642857,4.483333,4.577778
...,...,...,...,...,...,...,...,...,...,...
127,Южное Тушино,4.550000,4.320000,3.955000,4.368889,NaN,NaN,3.850000,4.650000,4.526667
128,Южнопортовый,4.250000,4.583333,3.922500,4.457447,5.000000,4.600000,4.220000,4.500000,4.361538
129,Якиманка,4.678571,5.000000,4.197959,4.408333,4.582353,4.866667,4.400000,4.650000,4.518182
130,Ярославский,4.700000,4.611111,3.995000,4.465909,NaN,4.000000,2.966667,4.757143,3.488889


Присоединим эти данные к нашей таблице.

In [ ]:
merged = merged.merge(gr, on='район', how='left')

In [ ]:
merged.to_csv('all.csv')

______________________________________________________________________________________________________________________________________________________